In [ ]:
#default_exp protonets
#export

from torchmeta.datasets.helpers import miniimagenet, omniglot
from torchmeta.utils.data import BatchMetaDataLoader
from torchmeta.utils.gradient_based import gradient_update_parameters
from torchmeta.modules import MetaModule, MetaSequential, MetaConv2d, MetaBatchNorm2d, MetaLinear

import kornia as K
import higher

import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchnet
import tqdm

from collections import OrderedDict
from copy import deepcopy

from unsupervised_meta_learning.nn_utils import Flatten, conv3x3, get_proto_accuracy

In [ ]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [ ]:
#export

def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

In [ ]:
#export

class ProtoNet(nn.Module):
    def __init__(self, encoder=None, **kwargs):
        super().__init__()
        self.encoder = encoder
        if encoder is None:
            x_dim = kwargs['x_dim']
            hid_dim = kwargs['hid_dim']
            self.encoder = nn.Sequential(
                conv_block(x_dim, hid_dim),
                conv_block(hid_dim, hid_dim),
                conv_block(hid_dim, hid_dim),
                conv_block(hid_dim, hid_dim),
                Flatten()
            )
        
    def loss(self, sample):
        xs = sample['train']
        xq = sample['test']

        # might have to change this bit to catch the correct shapes and sizes
        n_class = xs.shape[0]
        n_support = xs.shape[1]
        n_query = xq.shape[1]
        
        target_inds = torch.arange(0, n_class).view(n_class, 1, 1).expand(n_class, n_query, 1).long()
        
        if xq.is_cuda:
            target_inds = target_inds.to('cuda')
            
        x = torch.cat([
            xs.view(n_class * n_support, *xs.size()[2:] ),
            xq.view(n_class * n_query, *xq.size()[2:])
        ], 0)
        
        z = self.encoder(x)
        z_dim = z.size(-1)
        
        z_proto = z[:n_class * n_support].view(n_class, n_support, z_dim).long()
        zq = z[n_class * n_support:]
        
        dists = torch.cdist(zq, z_proto)
        
        log_p_y: torch.tensor = F.log_softmax(-dists, dim=1).view(n_class, n_query, -1)
        
        loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
        
        _, y_hat = log_p_y.max(2)
        
        acc_val = torch.eq(y_hat, target_inds.squeeze()).float().mean()
        
        return loss_val, {
            'loss': loss_val.item(),
            'acc': acc_val.item()
        }
        
        

In [ ]:
#export
class PrototypicalNetwork(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_size=64):
        super(PrototypicalNetwork, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.hidden_size = hidden_size

        self.encoder = nn.Sequential(
            conv3x3(in_channels, hidden_size),
            conv3x3(hidden_size, hidden_size),
            conv3x3(hidden_size, hidden_size),
            conv3x3(hidden_size, out_channels)
        )
    
    def forward(self, inputs):
        embeddings = self.encoder(inputs.view(-1, *inputs.shape[2:]))
        return embeddings.view(*inputs.shape[:2], -1)

In [ ]:
#export
def get_num_samples(targets, num_classes, dtype=None):
    batch_size = targets.size(0)
    with torch.no_grad():
        ones = torch.ones_like(targets, dtype=dtype)
        num_samples = ones.new_zeros((batch_size, num_classes))
        num_samples.scatter_add_(1, targets, ones)
    return num_samples

In [ ]:
#export 
def get_prototypes(emb, targets, num_classes):
    """Compute the prototypes (the mean vector of the embedded training/support 
    points belonging to its class) for each classes in the task.
    Parameters
    ----------
    embeddings : `torch.FloatTensor` instance
        A tensor containing the embeddings of the support points. This tensor 
        has shape `(batch_size, num_examples, embedding_size)`.
    targets : `torch.LongTensor` instance
        A tensor containing the targets of the support points. This tensor has 
        shape `(batch_size, num_examples)`.
    num_classes : int
        Number of classes in the task.
    Returns
    -------
    prototypes : `torch.FloatTensor` instance
        A tensor containing the prototypes for each class. This tensor has shape
        `(batch_size, num_classes, embedding_size)`.
    """
    
    batch_size, emb_size = emb.size(0), emb.size(-1)
    
    num_samples = get_num_samples(targets, num_classes, dtype=emb.dtype)
    num_samples.unsqueeze_(-1)
    num_samples = torch.max(num_samples, torch.ones_like(num_samples))
    
    prototypes = emb.new_zeros((batch_size, num_classes, emb_size))
    indices = targets.unsqueeze(-1).expand_as(emb)
    
    prototypes.scatter_add_(1, indices, emb).div_(num_samples)
    
    return prototypes

In [ ]:
#export
def prototypical_loss(prototypes, emb, targets, **kwargs):
    """Compute the loss (i.e. negative log-likelihood) for the prototypical 
    network, on the test/query points.
    Parameters
    ----------
    prototypes : `torch.FloatTensor` instance
        A tensor containing the prototypes for each class. This tensor has shape 
        `(batch_size, num_classes, embedding_size)`.
    embeddings : `torch.FloatTensor` instance
        A tensor containing the embeddings of the query points. This tensor has 
        shape `(batch_size, num_examples, embedding_size)`.
    targets : `torch.LongTensor` instance
        A tensor containing the targets of the query points. This tensor has 
        shape `(batch_size, num_examples)`.
    Returns
    -------
    loss : `torch.FloatTensor` instance
        The negative log-likelihood on the query points.
    """
    squared_distances = torch.sum(
        (prototypes.unsqueeze(2) - emb.unsqueeze(1)) ** 2, dim = -1
    )
    return F.cross_entropy(-squared_distances, targets, **kwargs)

In [ ]:
#export
def proto_train(model, folder="./data/", num_shots=1, num_ways=5, batch_size=16, device='cuda', num_batches=100):
    ds = omniglot(folder,
                  shots=num_shots,
                  ways=num_ways,
                  shuffle=True,
                  test_shots=15,
                  meta_train=True,
                  download=True
                 )
    dataloader = BatchMetaDataLoader(ds,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=4
                                    )
    model.to(device)
    model.train()
    
    optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    with tqdm.notebook.tqdm(dataloader, total=num_batches) as pbar:
        for batch_idx, batch in enumerate(pbar):
            model.zero_grad()

            train_inputs, train_targets = batch['train']
            train_inputs, train_targets = train_inputs.to(device), train_targets.to(device)
            
            train_emb = model(train_inputs)
            
            test_inputs, test_targets = batch['test']
            test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
            test_emb = model(test_inputs)
            
            prototypes = get_prototypes(train_emb, train_targets, ds.num_classes_per_task)
            loss = prototypical_loss(prototypes, train_emb, train_targets)
            
            loss.backward()
            optimiser.step()
            
            with torch.no_grad():
                acc = get_proto_accuracy(prototypes, test_emb, test_targets)
                pbar.set_postfix(accuracy='{0:.4f}'.format(acc.item()))
                
            if batch_idx >= num_batches:
                break

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 01_nn_utils.ipynb.
Converted 02_maml.ipynb.
Converted 03_protonet.ipynb.
Converted 04_cactus.ipynb.
Converted index.ipynb.


In [ ]:
model = PrototypicalNetwork(1, 64, hidden_size=64)
proto_train(model)

  0%|          | 0/100 [00:00<?, ?it/s]

/home/ojass/anaconda3/envs/jax/lib/python3.9/site-packages/torchvision/transforms/functional.py:942: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ojass/anaconda3/envs/jax/lib/python3.9/site-packages/torchvision/transforms/functional.py:942: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ojass/anaconda3/envs/jax/lib/python3.9/site-packages/torchvision/transforms/functional.py:942: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ojass/anaconda3/envs/jax/lib/python3.9/site-packages/torchvision/transforms/functional.py:942: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
